# How diverse are archaeal histones?
## Approach 
To look at the diversity of histones in archaea we need to:
0. Collect all archaeal proteins. (will need to install NCBI's eDirect)
1. Identify which proteins are histones.
2. Create database having at least gene name, sequence, genomic GC, and type of phile (use keywords and look through each species). 
3. Vizualize these histones on 3D scatterplot (length vs genomic GC vs pI).
4. Use half_blast to identify doublet proteins.
5. Use sklearn to cluster proteins:
    1. 3 dimensions: length, genomic GC, pI.
    2. n dimensions: 3-mer(or other mer of aligned protiens).
    3. n dimensions: each amino acid of aligned protiens.
6. Evaluate each cluster:
    1. Find average or median protein from each cluster.
    2. See what are the characteristic of each group.
    3. Figure out which phyla are represented in each cluster.

#### 0. Collect all archaeal proteins. (will need to install NCBI's eDirect)

In [3]:
!esearch -db protein -query "archaea[organism]" | efetch -format fasta > archaeal_proteins.fa

500 Can't connect to eutils.be-md.ncbi.nlm.nih.gov:443 (Operation timed out)
No do_post output returned from 'https://eutils.be-md.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&query_key=1&WebEnv=NCID_1_2945410_130.14.22.76_9001_1575404443_1831832940_0MetA0_S_MegaStore&rettype=fasta&retmode=text&retstart=1721500&retmax=500&edirect_os=darwin&edirect=12.8&tool=edirect&email=shawn@Shawns-MacBook-Pro.local'
Result of do_post http request is
$VAR1 = bless( {
                 '_content' => 'Can\'t connect to eutils.be-md.ncbi.nlm.nih.gov:443 (Operation timed out)

LWP::Protocol::https::Socket: connect: Operation timed out at /System/Library/Perl/Extras/5.18/LWP/Protocol/http.pm line 51.
',
                 '_request' => bless( {
                                        '_headers' => bless( {
                                                               'content-type' => 'application/x-www-form-urlencoded',
                                                               'user-agent' =>

500 Can't connect to eutils.be-md.ncbi.nlm.nih.gov:443 (Bad hostname)
No do_post output returned from 'https://eutils.be-md.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&query_key=1&WebEnv=NCID_1_2945410_130.14.22.76_9001_1575404443_1831832940_0MetA0_S_MegaStore&rettype=fasta&retmode=text&retstart=2698500&retmax=500&edirect_os=darwin&edirect=12.8&tool=edirect&email=shawn@cu-vpn-colorado-edu-198.11.28.9.int.colorado.edu'
Result of do_post http request is
$VAR1 = bless( {
                 '_content' => 'Can\'t connect to eutils.be-md.ncbi.nlm.nih.gov:443 (Bad hostname)

LWP::Protocol::https::Socket: Bad hostname \'eutils.be-md.ncbi.nlm.nih.gov\' at /System/Library/Perl/Extras/5.18/LWP/Protocol/http.pm line 51.
',
                 '_request' => bless( {
                                        '_uri' => bless( do{\(my $o = 'https://eutils.be-md.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi')}, 'URI::https' ),
                                        '_headers' => bless( {
             

Retrying efetch, step 2: Unable to obtain query #1
Retrying efetch, step 2: Unable to obtain query #1
Retrying efetch, step 2: Unable to obtain query #1
ERROR in efetch (2701001-2701500 / 6316319): Unable to obtain query #1
Replicate for debugging with:
  edirutil -db protein -web NCID_1_2945410_130.14.22.76_9001_1575404443_1831832940_0MetA0_S_MegaStore -key 1 -count 6316319 -step 1 -seconds 73717.536 | efetch -format fasta -start 2701001 -stop 2701500
Retrying efetch, step 2: Unable to obtain query #1
Retrying efetch, step 2: Unable to obtain query #1
Retrying efetch, step 2: Unable to obtain query #1
ERROR in efetch (2701501-2702000 / 6316319): Unable to obtain query #1
Replicate for debugging with:
  edirutil -db protein -web NCID_1_2945410_130.14.22.76_9001_1575404443_1831832940_0MetA0_S_MegaStore -key 1 -count 6316319 -step 1 -seconds 73728.297 | efetch -format fasta -start 2701501 -stop 2702000
Retrying efetch, step 2: Unable to obtain query #1
Retrying efetch, step 2: Unable to 

#### 1. Identify which proteins are histones. (we can use hmmscan to do this)